In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import numpy as np
import anndata as ad
import pandas as pd
import scanpy as sc
import random
import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle
from utils import regression_model

np.random.seed(1024) 
random.seed(2048)
tf.random.set_seed(1024)

Prepare training dataset

In [10]:
emb_size = 32

Leave 3 genes 'Rps6', 'Zfp292', 'Prdm1' for validation of model performance in step 3.

In [11]:
prop = pd.read_csv("data/training_cell_state_proportions.csv", index_col=0)
prop = prop.drop(index=['Rps6', 'Zfp292', 'Prdm1'])

In [12]:
g_embed = np.load(f'submission/embedding/gene_embedding_{emb_size}.npy')
g_name = np.load(f'submission/embedding/gene_names_{emb_size}.npy', allow_pickle= True)

genes = list(set(prop.index.tolist()))
training_conditions = [g for g in genes if g in g_name]

#extract embedding of training set
X, Y = [], []
for g in training_conditions:
    x = g_embed[list(g_name).index(g)]
    y = np.array(prop.loc[g].values)
    X.append(x)
    Y.append(y)

X = np.array(X)
Y = np.array(Y)

train different model to map 32D gene embedding vector to 5D output vector

In [7]:
#scaler on the whole gene list
X_scaler = StandardScaler()
X_scaler.fit(g_embed)

with open(f'./submission/scaler_{emb_size}.pkl', 'wb') as f:
    pickle.dump(X_scaler, f)

In [ ]:
epochs=2000

X, Y = shuffle(X,Y,random_state=36)
kf = KFold(n_splits=10)
kfold = kf.split(X, Y)

for n_try in range(8):

    score = []
    predictions = []

    for k, (train, val) in enumerate(kfold):
        X_train, Y_train = X[train], Y[train]
        X_val, Y_val = X[val], Y[val]

        X_train_transformed = X_scaler.transform(X_train)
        X_val_transformed = X_scaler.transform(X_val)

        for dropout_rate in [0]:
            for l1 in [1e-3, 1e-4,0]:
                for l2 in [0, 1e-5]:
                    for learning_rate in [1e-3]:

                        #define lr schedule
                        decay = learning_rate / (epochs-50) * 10                        
                        def scheduler(epoch, lr):
                            if epoch < 50:
                                return lr
                            else:
                                return lr * 1/(1+decay*epoch) #tf.math.exp(-0.1)

                        for mae_weight in [2]:
                            for batch_size in [8,16]:
                                model_path = f"./submission/checkpoints/NN/model_{emb_size}_{dropout_rate}_{l1}_{l2}_{learning_rate}_{mae_weight}_{batch_size}_{k}_{n_try}.h5"

                                #define model
                                model = regression_model(
                                            input_size = emb_size,
                                            output_size = 5,
                                            size_dense = 8,
                                            dropout_rate = dropout_rate,
                                            l1 = l1,
                                            l2 = l2,
                                            learning_rate = learning_rate,
                                            mae_weight=mae_weight
                                        )

                                #define callbacks
                                checkpoint = keras.callbacks.ModelCheckpoint(
                                    model_path,
                                    monitor='val_mae', 
                                    verbose=0, save_best_only=True, mode='min'
                                )
                                early_stop = keras.callbacks.EarlyStopping(monitor='val_mae', patience=200, mode='min')
                                schedule = keras.callbacks.LearningRateScheduler(scheduler, verbose=0)

                                # training
                                model.fit(X_train_transformed, Y_train, 
                                            # sample_weight=w, 
                                            epochs=epochs, 
                                            shuffle=True, 
                                            batch_size=batch_size,
                                            validation_data=(X_val_transformed,Y_val),
                                            callbacks = [checkpoint, early_stop, schedule],
                                            verbose = 0
                                            )

                                # calculate best val_mae
                                model.load_weights(model_path)
                                mae_train = mean_absolute_error(Y_train, model.predict(X_train_transformed))
                                mae_val = mean_absolute_error(Y_val, model.predict(X_val_transformed))


                                score.append([
                                    n_try, k, dropout_rate, l1, l2, learning_rate, mae_weight, batch_size, mae_train, mae_val
                                ])

                                print(n_try, k, dropout_rate, l1, l2, learning_rate, mae_weight, batch_size, mae_train, mae_val)

                                del model, model_path


    score = pd.DataFrame(
        score,
        columns = ["n_try", "k", "dropout_rate", "l1", "l2", "learning_rate", "mae_weight", "batch_size", "mae_train", "mae_val"]
    )
    score.to_csv(f"./submission/predictions/NN_{emb_size}_score_try{n_try}.csv")